# Cleanlab
## Treating the Labels as Noisy
### Learning from Noisy Labels
Link: https://github.com/cgnorthcutt/cleanlab/

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
train = pd.read_json('train.json')
train.head()

,sentiment,text,uid
0,negative,@ AdilNisarButt pakistan ka ghra tauq he Pakis...,3
1,negative,Madarchod mulle ye mathura me Nahi dikha tha j...,41
2,positive,@ narendramodi Manya Pradhan Mantri mahoday Sh...,48
3,positive,@ Atheist _ Krishna Jcb full trend me chal rah...,64
4,positive,@ AbhisharSharma _ @ RavishKumarBlog Loksabha ...,66


In [3]:
num_classes = len(set(list(train['sentiment'])))

# Refering to : [Twitter-Airlines](https://github.com/martinpella/twitter-airlines/blob/master/shallow_learning.ipynb) for cleaning data. 

In [4]:
from utils import TextCleaner, CleanTwitter

In [ ]:
X_train, X_test, y_train, y_test, tfidf_train, tfidf_test = CleanTwitter(train)

# LearningWithNoisyLabels : [IrisSimple](https://github.com/cgnorthcutt/cleanlab/blob/master/examples/iris_simple_example.ipynb)?

In [ ]:
from cleanlab.classification import LearningWithNoisyLabels
from cleanlab.noise_generation import generate_noise_matrix_from_trace
from cleanlab.noise_generation import generate_noisy_labels
from cleanlab.util import value_counts
from cleanlab.latent_algebra import compute_inv_noise_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
m = LogisticRegression(C=4, dual=True)
rp = LearningWithNoisyLabels(clf = m)

In [ ]:
# Not sure if this is how you do it? HOW TO CHANGE THESE VALUES?

# Set the sparsity of the noise matrix.
FRAC_ZERO_NOISE_RATES = 0.5
# A proxy for the fraction of labels that are correct.
avg_trace = 0.85 # ~15% wrong labels. Increasing makes the problem easier.
py = value_counts(y_train)
noise_matrix = generate_noise_matrix_from_trace(
        K = num_classes,
        trace = num_classes * avg_trace, 
        py = py,
        frac_zero_noise_rates = FRAC_ZERO_NOISE_RATES,
    )

# Create noisy labels
#s = generate_noisy_labels(y_train, noise_matrix)

In [ ]:
print('WITHOUT confident learning,', end=" ")
m = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
_ = m.fit(tfidf_train, y_train)
pred = m.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test), 2))

WITHOUT confident learning, test accuracy: 0.62


In [ ]:
print('WITH confident learning (without noise matrix given),', end=" ")
_ = rp.fit(tfidf_train, y_train)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),2))

WITH confident learning (without noise matrix given), 

/Users/meghanabhange/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/meghanabhange/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


test accuracy: 0.61


In [ ]:
print('WITH confident learning (noise matrix given),', end=" ")
_ = rp.fit(tfidf_train, y_train, noise_matrix=noise_matrix)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),2))


WITH confident learning (noise matrix given), 

/Users/meghanabhange/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/meghanabhange/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


test accuracy: 0.62


In [ ]:
print('WITH confident learning (noise / inverse noise matrix given),', end=" ")
_ = rp.fit(tfidf_train, y_train, noise_matrix = noise_matrix, inverse_noise_matrix=compute_inv_noise_matrix(py, noise_matrix))
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),2))

WITH confident learning (noise / inverse noise matrix given), 

/Users/meghanabhange/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/meghanabhange/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


test accuracy: 0.61


In [ ]:
print('WITH confident learning (using latent noise matrix estimation),', end=" ")
m = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
rp = LearningWithNoisyLabels(clf = m)
_ = rp.fit(tfidf_train, y_train)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),2))

In [ ]:
print('WITH confident learning (using calibrated confident joint),', end=" ")
m = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
rp = LearningWithNoisyLabels(clf=m)
_ = rp.fit(tfidf_train, y_train)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),2))